# AIR for QKD Based on Shortened Polar Codes

> Reference: 
> 1. Zhou, H. et al. Appending Information Reconciliation for Quantum Key Distribution. Preprint at https://doi.org/10.48550/arXiv.2204.06971 (2022).
> 2. Bioglio, V., Gabry, F. & Land, I. Low-Complexity Puncturing and Shortening of Polar Codes. Preprint at https://doi.org/10.48550/arXiv.1701.06458 (2017).
> 3. Tal, I. & Vardy, A. How to Construct Polar Codes. Preprint at https://doi.org/10.48550/arXiv.1105.6164 (2013).
> 4. Cassagne, A. et al. AFF3CT: A Fast Forward Error Correction Toolbox! Elsevier SoftwareX 10, 100345 (2019). https://github.com/aff3ct/aff3ct

**Short summary: Increase the size of frozen vector until reconciliation success.**

Suppose Alice and bob are going to use polar codes to do Information Reconciliation (IR) for Quantum Key Distribution (QKD). 
Before starting the IR process, they need to: 

1. Test the performance of quantum channels, obtain the estimated quantum bit error rate (QBER) $p_{QBER}$;
2. Configure the length of sifted KEY $L$, then use $N = 2^{\lceil \log_2 L \rceil}$ as the length of polar codes;
3. Setup the efficiency $f$ of IR scheme, which can be calculated by 
   
   $$ f = \frac{N-K}{H_2(p_{QBER}) \times L}, $$

   where $K$ is the number of information bits of polar codes, $H_2$ is the binary entropy function. 
   With configured efficiency $f$, Alice and Bob can decide $K$;
4. Generate frozen vector for configured parameter $N, K$ and $p_{QBER}$. Specifically, shorten the polar codes to a $(L, K)$ one via the [Bioglio-Gabry-Land (BGL) method](http://arxiv.org/abs/1701.06458). Then froze $L - K$ bits by the reliability of bit channel calculated by the methos proposed in [_How to Construct Polar Codes_](https://doi.org/10.1109/TIT.2013.2272694);

Now, at the begining of IR process, let Alice and Bob hold two bit strings as their sifted KEY $K_s^A$ and $K_s^B$, respectively. 
The IR is then conducted as follows: 

1. Alice fills the $L$-length bit string $K_s^A$ into the non-frozen positions of an all-zero $N$-length bit string, obtain $\hat{K}_s^A$; Similarly, Bob use his sifted KEY $K_s^B$ to obtain $\hat{K}_s^B$;
2. Alice feeds $\hat{K}_s^A$ to the polar encoder. 
   The polar encoder will apply the polar transformation $G^{\log_2 N} = T^{\otimes \log_2 N}$, return $u_A = \hat{K}_s^A \cdot T^{\otimes \log_2 N}$, while the shortened bits will remain 0 by the design of the shortening method. 
   Note that typically there will be bit-reversal operation after the polarization, but here we omit it for simplicity (this only differs from Arikan's original polar codes in the ordering of the code bits), and $T$ is the polarization kernel;
3. Alice calculates the CRC tag $T_A = \mathrm{CRC}\left( u^A \right)$, then sents it to Bob via public channel;
4. Alice extracts extracts the frozen bits of $u_A$ according to the frozen vector, obtains syndrome bit string $S_A$;
5. Alice send $S_A$ through the public channel;
6. Bob feeds $\hat{K}_s^B$ as codeword and syndrome $S_A$ as frozen bits to the polar decoder, corrects the error in $\hat{K}_s^B$ and obtain $u_B$;
7. Bob applies CRC check to $u_B$, and obtains CRC tag $T_B = \mathrm{CRC}(u_B)$;
8. Bob $T_A$ and $T_B$, if they are the same, then Bob publicly announces that the IR process is successful. 

This one-shot CRC-checked polar codes-based IR scheme has some drawbacks: 
- **Low flexibility on sifted KEY length $L$**: For different sifted KEY length $L$, Alice and Bob needs lots of experiments or simulation to find a reasonable efficiency, otherwise high failure rate or low secret KEY yield will be observed; 
- **Dichomotic results**: If the reconciliation process fails, then the sifted KEY will be totally wasted, no other way to reuse the leaked privacy as error syndrome.

_Appending Information Reconciliation_ attacks this problem by introducing a multi-stage polar code reconciliation protocol, which is depicted in the following figure in [AIR paper](https://doi.org/10.48550/arXiv.2204.06971). 

![Figure-1 in AIR Paper](./images/fig_ref_2.png)

Before the QKD process, Alice and Bob agree with the number of stages of reconciliation, $R_m$, as well as the reconciliation efficiency of each stage. 
The reconciliation efficiency increases as reconciliation proceeds to the next stage, since the frozen bits is selected by bit-channel reliability, the corresponding set of frozen bits is always contained in the one of the next stage. 
The reconciliation will stop at any time when reconciliation is judged to be success -- which is checked and announced by Bob via CRC.
In the end of $k$-th stage, if the CRC check indicates failure of reconciliation, then Bob will announce this result -- tell Alice by sending the flag $\sigma = 1$ through the public channel. 
With $\sigma = 1$, Alice appends additional syndrome by selecting the bits in $u_A$ of position $\mathcal{F}_{k+1} \oplus \mathcal{F}_{k}$, i.e. the additional frozen bits of next stage, and sending them through public channel. 
Now Bob can do reconciliation with new error syndrome appended to the one of former stages. 

AIR can hence have a very low failure rate, as long as the configured efficiency of the last stage is high enough. 
Also, AIR can avoid the average efficiency to be too large because of the adaptive reconciliation procedure -- most sifted key pairs can be successfully reconciled with a low efficiency. 

### Import Necessary Libs

We use a modified version of py_aff3ct library to implement polar codes, which can be found in `submodules` folder. 
Make sure you have compiled the py_aff3ct in the corresponding directory.
Customized py_aff3ct modules are implemented in the `src` directory. 

In [1]:
# Import necessary libraries
import sys
sys.path.insert(0, '../submodules/py_aff3ct/build/lib')
import numpy as np
import time

import py_aff3ct

### Read Dataset

Alice and Bob's sifted KEYs are stored in binary files. 
Here we read them via numpy.

In [2]:
with open('../data/Alice_key.bin', 'rb') as f: 
    data = np.fromfile(f, dtype = np.uint8)
    pass

alice_sifted_key = np.unpackbits(data)

with open('../data/Bob_key.bin', 'rb') as f: 
    data = np.fromfile(f, dtype = np.uint8)
    pass

bob_sifted_key = np.unpackbits(data)

alice_sifted_key = alice_sifted_key.astype(np.int32)
bob_sifted_key = bob_sifted_key.astype(np.int32)

assert alice_sifted_key.shape[0] == bob_sifted_key.shape[0], "sifted key should of equal length."

### Set up Parameters

- **num_samples**: The number of sampled pairs of sifted KEYs in the simulation;
- **q_ber**: The polar code assumed bit error rate;
- **scl_len**: Length of polar SCL decoder list;
- **crc_size**: The length of CRC tag.

In [3]:
# seed = 1234
num_samples = 1000

len_sifted_key = alice_sifted_key.shape[0]
# len_sifted_key = 65536
# alice_sifted_key = alice_sifted_key[:len_sifted_key]
# bob_sifted_key = bob_sifted_key[:len_sifted_key]

# Polar codes parameters
# Assumed bit error rate
q_ber = 0.0345

# Polar code Block size
# The polar code block size can be determined automatically
n = np.ceil(np.log2(len_sifted_key)).astype(np.int64)
block_encoded_bits = 2 ** n
# The length of the list of SCL Decoder will affect the performance of the SCL Decoder ---- and running time of codes
scl_len = 16

# CRC check
crc_size = 32

# Quantum bit error rate from experimental data
q_ber_stat = np.logical_xor(alice_sifted_key, bob_sifted_key).astype(np.int32).sum() / len_sifted_key

print("Quantum Bit Error Rate (QBER) from dataset: {} / {} = {}".format(
    np.logical_xor(alice_sifted_key, bob_sifted_key).astype(np.int32).sum(), len_sifted_key, q_ber_stat))
print("*"*32)
print("Configured QBER: {}".format(q_ber))
if np.abs(q_ber - q_ber_stat) > 0.001:
    print("*"*32)
    print("Deviation between assumed QBER and statistical QBER: {}".format(np.abs(q_ber - q_ber_stat)))
    print("*"*32)

Quantum Bit Error Rate (QBER) from dataset: 3813 / 105976 = 0.03597984449309278
********************************
Configured QBER: 0.0345
********************************
Deviation between assumed QBER and statistical QBER: 0.001479844493092776
********************************


### Estimation of Assumed QBER

In proposed protocol, configured QBER $p_{QBER}$ should be decided before the reconciliation phase, empirically. But we have no additional experiment data to obtain this empirical value here. Therefore, strictly speaking, we did not implement a complete QKD protocol. It becomes meaningful to consider the case that $p_{QBER}$ is deviated from the real QBER $p_{real}$, this could happen when only very limited resources are available for estimating $p_{real}$ before the reconciliation phase. 

Suppose we use sifted keys data collected from experiments of length $L = 105976$ to obtain an estimation $\hat{p}_{real}$ of $p_{real}$, the $99\%$ confidence interval (by the normal approximation of binomial distribution or by adopting the Bayesian posterior with $\text{Beta}(1,1)$ as prior) of $\hat{p}_{QBER}$ will be contained in interval $[0.0345, 0.0375]$. Then we can test the IR scheme, using polar codes optimized for $p_{QBER} = 0.036 \pm 0.0015$ (i.e. `q_ber` in codes) and on simulated random samples obeys $p_{real} \approx 0.035980$ (i.e. `q_ber_stat` in codes). The confidence interval will be larger for less samples. For example, consider $\hat{p}_{real}$ estimated from $10000$ pairs of bit obeys error rate of $\hat{p}_{real} = 0.036$. The $95\%$ confidence interval will be approximately $[0.032, 0.04]$. 
Such endpoint values can serve as values of $p_{QBER}$ (`p_ber` in codes) to test the worst-case performance of this IR scheme under such extreme conditions.

### Efficiency

Note that, in this realization, when the sifted KEYs are shorter than polar codes, the polar codes will be shortened. 
Specifically, for sifted KEYs of length $L$, a $(N, K)$ polar code will be shortened to a $(L, K)$ polar code. 
This implies that only $L - K$ frozen bits (of information) will be leaked to reconcile $L \times H_2(p_{QBER})$ bits of error information.
Hence the reconciliation efficiency (with single stage and no CRC tag) is still: 

$$f = \frac{L - K}{L \times H_2(p_{QBER})}.$$

However, since AIR is an adaptive IR scheme, the efficiency can only be estimated from collected data. 
More specifically, suppose the program is configured with $R_m = 4$ stages with efficiency $\{f_i\}_{i=1}^{R_m}$, and for $Q$ simulation samples, the failure of each stage is $\{ E_i \}_{i=1}^{R_m}$, then the overall efficiency is calculated as

$$f = \sum_{i=1}^{R_m} \frac{(E_{i-1} - E_{i})f_i}{Q}.$$

Here $E_{0} = Q$. 

### Configure the efficiency

- **configured_efficiency_stage**: A numpy array which decides the efficiency of each stage, which include the gain introduced by CRC tag. 

In [4]:
shannon_H2 = lambda x: -x * np.log2(x) - (1 - x) * np.log2(1 - x)

ideal_frozen_bits = len_sifted_key * shannon_H2(q_ber_stat)

crc_efficiency_gain = crc_size / ideal_frozen_bits

configured_efficiency_stage = np.asarray([1.18, 1.19, 1.21, 1.25], dtype = np.float64)
# For reconciling experiment data, finer efficiency stages should be configured to obtain better efficiency performance
# configured_efficiency_stage_experiment = np.arange(1.12, 1.152, 0.002, dtype = np.float64)
# configured_efficiency_stage = configured_efficiency_stage_experiment
# The following efficiency stage can be used to step-by-step check the codes
# configured_efficiency_stage = np.asarray([1.03, 1.25], dtype = np.float64)
efficiency_stage = configured_efficiency_stage - crc_efficiency_gain

effective_frozen_bits_stage = np.asarray(efficiency_stage * ideal_frozen_bits, dtype = np.int64)
effective_efficiency_stage = effective_frozen_bits_stage.astype(np.float64) / ideal_frozen_bits

# The number of shortened bits will be the same in all stages
shortened_bits = block_encoded_bits - len_sifted_key
info_bits_stage = block_encoded_bits - effective_frozen_bits_stage - shortened_bits

print('-' * 32)
print('FORWARD RECONCILICATION PHASE SIMULATION')
print('Length of Sifted KEY: {} \nLength of Polar Codes: {}'.format(len_sifted_key, block_encoded_bits))
print('----> Number of Shortened Bits: {}'.format(shortened_bits))
print('Statistical Qubit Error Rate: {} -> H_2 = {}'.format(q_ber_stat, shannon_H2(q_ber_stat)))
print('----> Estimated Quantum Channel Leaked Bits (L * H_2(QBER)): {}'.format(len_sifted_key * shannon_H2(q_ber_stat)))
print('Configured Qubit Error Rate: {} -> H_2 = {}'.format(q_ber, shannon_H2(q_ber)))
print('----> Estimated Least Polar Code Frozen Bits (N - K): {}'.format((len_sifted_key) * shannon_H2(q_ber)))
print("-"*32)
print("Configured Efficiency stages: {}".format(configured_efficiency_stage))
# print("Configured Efficiency stages: {}".format(efficiency_stage))
print("-"*32)
print("Effecctive Efficiency Stages (without CRC tag): {}".format(effective_efficiency_stage))
print("Number of bits leaked by CRC Tag: {}. Efficiency gain: {}".format(crc_size, crc_efficiency_gain))
print("----> Effective Efficiency Stages: {}".format(effective_efficiency_stage + crc_efficiency_gain))
print('-' * 32)
print('Enc-Dec Effective Information Bits Stages: {}'.format(info_bits_stage))
print('Enc-Dec Effective Frozen Bits Stages:      {}'.format(effective_frozen_bits_stage))
print('-' * 32)
# print('MotherCode Efficiency Stages: {} / {} = {}'.format(info_bits_stage, block_encoded_bits * shannon_H2(q_ber), info_bits_stage / (block_encoded_bits * shannon_H2(q_ber))))

--------------------------------
FORWARD RECONCILICATION PHASE SIMULATION
Length of Sifted KEY: 105976 
Length of Polar Codes: 131072
----> Number of Shortened Bits: 25096
Statistical Qubit Error Rate: 0.03597984449309278 -> H_2 = 0.2235460591884705
----> Estimated Quantum Channel Leaked Bits (L * H_2(QBER)): 23690.51716855735
Configured Qubit Error Rate: 0.0345 -> H_2 = 0.21647981144138728
----> Estimated Least Polar Code Frozen Bits (N - K): 22941.66449731246
--------------------------------
Configured Efficiency stages: [1.18 1.19 1.21 1.25]
--------------------------------
Effecctive Efficiency Stages (without CRC tag): [1.17861505 1.18861905 1.20862706 1.24864307]
Number of bits leaked by CRC Tag: 32. Efficiency gain: 0.0013507514324115812
----> Effective Efficiency Stages: [1.1799658  1.1899698  1.20997781 1.24999382]
--------------------------------
Enc-Dec Effective Information Bits Stages: [78054 77817 77343 76395]
Enc-Dec Effective Frozen Bits Stages:      [27922 28159 28633 

### Generate Frozen Bits and Shortened Bits

The frozen bits are union of two sets: Shortened bits and 'effective' frozen bits. 
The former will be fixed to 0 for Alice and Bob in advance. Hence will not be sent through the public channel. They are generated via the BGL method introduced in [_Low-Complexity Puncturing and Shortening of Polar Codes_](http://arxiv.org/abs/1701.06458).

Effective frozen bits work as the error syndrome in the reconciliation protocol, which are selected according to the bit channels' reliability. 
The reliability is calculated via the method introduced in [_How to Construct Polar Codes_](https://doi.org/10.1109/TIT.2013.2272694).

Specifically, the channel reliability in `conf/polar/bsc/` are generated by programs of two repositories: A C++ implementation [PolarCodeForQKD](https://github.com/cfxtby/PolarCodeForQKD) and a MATLAB program [PolarCodes-Encoding-Decoding-Construction](https://github.com/YuYongRun/PolarCodes-Encoding-Decoding-Construction). Two programs produces the same results (while the C++ program is much faster) and possible difference between the final sort of channels according to error rate is caused by floating-point storage and calculation errors.

> Remark: Use `outf << std::scientific << std::setprecision(std::numeric_limits<double>::max_digits10);` in the C++ program to losslessly save and read double-precision variables.

Since the frozen bits are selected by the channel reliability, for non-decreasing efficiency of stages the set of frozen bits of one stage will always be a subset of the set of frozen bits of the next stage. This allows Alice to 'append' new error syndrome to Bob. 

In [5]:
sys.path.insert(0, '../src/')
from utils import generate_shortened_BGL, generate_frozen_position_from_file
from copy import deepcopy

fb_file_dir = '../conf/polar/bsc/n{}_bsc_ber{}_mu40.pc'.format(int(np.log2(block_encoded_bits)), q_ber)

# import py_aff3ct.tools.frozenbits_generator as tool_fb
# import py_aff3ct.tools.noise as tool_noise
# fbgen_file = tool_fb.Frozenbits_generator_file(block_info_bits - shortened_bits, block_encoded_bits, fb_file_dir)
# noise = tool_noise.Event_probability(q_ber)
# fbgen_file.set_noise(noise)
# frozen_bits_file = fbgen_file.generate()

try:
    with open(fb_file_dir, 'r') as f:
        lines = f.readlines()
        # The 4th line contains the reliability information
        reliability_line = lines[3].strip()
        # Convert to np.int64 array
        reliability_sorted_idx = np.array([int(x) for x in reliability_line.split()], dtype=np.int64)
except Exception:
    print("polar code .pc file not found")
    print("try to load _cpp.pc file")
    fb_file_dir = '../conf/polar/bsc/n{}_bsc_ber{}_mu40_cpp.pc'.format(int(np.log2(block_encoded_bits)), q_ber)
    with open(fb_file_dir, 'r') as f:
        lines = f.readlines()
        # The 4th line contains the reliability information
        reliability_line = lines[3].strip()
        # Convert to np.int64 array
        reliability_sorted_idx = np.array([int(x) for x in reliability_line.split()], dtype=np.int64)
shortened_idx = generate_shortened_BGL(block_encoded_bits, shortened_bits)
shortened_vec = np.zeros(block_encoded_bits, dtype=bool)
shortened_vec[shortened_idx] = np.ones_like(shortened_vec[shortened_idx])

frozen_vec_stage = []
effective_frozen_vec_stage = []

for effective_frozen_bits_nums in effective_frozen_bits_stage: 
    _, __, effective_frozen_idx = generate_frozen_position_from_file(fb_file_dir, effective_frozen_bits_nums, shortened_idx)
    frozen_bits = np.zeros(block_encoded_bits, dtype=bool)
    effective_frozen_bits = np.zeros(block_encoded_bits, dtype=bool)
    frozen_bits[shortened_idx] = np.ones_like(frozen_bits[shortened_idx]).astype(np.bool)
    frozen_bits[effective_frozen_idx] = np.ones_like(frozen_bits[effective_frozen_idx]).astype(np.bool)
    effective_frozen_bits[effective_frozen_idx] = np.ones_like(effective_frozen_bits[effective_frozen_idx]).astype(np.bool)
    frozen_vec_stage.append(deepcopy(frozen_bits))
    effective_frozen_vec_stage.append(deepcopy(effective_frozen_bits))
    pass

print("Number of Total Frozen Bits: {}".format([int(a.astype(np.int32).sum()) for a in frozen_vec_stage]))
print("Number of Effective Frozen Bits: {}".format([int(a.astype(np.int32).sum()) for a in effective_frozen_vec_stage]))
print("Differences:")
for i in range(len(frozen_vec_stage)-1):
    print('{} to {} : {}'.format(i, i+1, np.logical_xor(frozen_vec_stage[i], frozen_vec_stage[i+1]).astype(np.int64).sum()))

polar code .pc file not found
try to load _cpp.pc file
Totally 27922 bits appended to frozen bits set
Totally 28159 bits appended to frozen bits set
Totally 28633 bits appended to frozen bits set
Totally 29581 bits appended to frozen bits set
Number of Total Frozen Bits: [53018, 53255, 53729, 54677]
Number of Effective Frozen Bits: [27922, 28159, 28633, 29581]
Differences:
0 to 1 : 237
1 to 2 : 474
2 to 3 : 948


### Polar Encoder and CRC-Aided SCL Decoder

The polar codec is modified from the `py_aff3ct` library. 
Specifically: 
- The `light_encode` method of encoder is added to implement the polar transform;
- The `set_crc_const` method of decoder is implemented for set up the CA-SCL decoding CRC tag. 
- The `decode_siho_cw_flexible_frozen` method of decoder is implemente for CA-SCL decoding with arbitrary frozen bits values.

All modifications are implemented in the `aff3ct` library of `py_aff3ct`. 

In [6]:
from py_aff3ct.module.encoder import Encoder_polar
from py_aff3ct.module.decoder import Decoder_polar_SCL_naive_CA, Decoder_polar_SCL_naive
from py_aff3ct.module.crc import CRC, CRC_polynomial
crc_stage = []
# Since only light_encode will be performed, the number of information bits does not matter at all
# enc_alice = Encoder_polar(info_bits_stage[0], block_encoded_bits, frozen_vec_stage[stage])
# The frozen bits is useless in light_encode method
# enc_alice.set_frozen_bits(frozen_vec_stage[0])
enc_stage = []
dec_stage = []

for stage in range(len(effective_frozen_bits_stage)):
    crc_stage.append(CRC_polynomial(block_encoded_bits, '0x04C11DB7', crc_size))
    # The encoder works as a 'light encoder' to encode the (padded) alice sifted key into the codeword 
    enc_stage.append(Encoder_polar(info_bits_stage[stage], block_encoded_bits, frozen_vec_stage[stage]))
    dec_stage.append(Decoder_polar_SCL_naive_CA(info_bits_stage[stage], block_encoded_bits, scl_len, frozen_vec_stage[stage], crc_stage[stage]))
    enc_stage[stage].set_frozen_bits(frozen_vec_stage[stage])
    dec_stage[stage].set_frozen_bits(frozen_vec_stage[stage])
    pass

### LLR Calculater and Shortener

The error between Alice and Bob's sifted KEYs can be described by the binary symmetric channel (BSC) with event probability of quantum BER. 
Log-likelihood ratio (LLR) of the output $x \in \{0, 1\}$ of BSC with error rate $p$ can be calculated by

$$ LLR_p(x) = (1 - 2 x) \ln \left( \frac{1-p}{p} \right). $$

For each stage, a LLR calculater will be created to calculate the LLR of the output of Alice's polar encoder. 
Also, a shortener will be created to set the LLR of shortened bits of the input frozen vector of Bob's polar decoder to a very large value (50, for example).

In [7]:
from utils import llr_calculater_bsc, shortener

llr_bsc_stage = []
shortener_stage = []

for stage in range(len(frozen_vec_stage)):
    llr_bsc_stage.append(llr_calculater_bsc(q_ber, block_encoded_bits))
    shortener_stage.append(shortener(block_encoded_bits))
    # All shortener takes the same shortened positions
    shortener_stage[stage].set_shorten_bits(deepcopy(shortened_vec.astype(np.float32).reshape((1, -1))))
    pass

### Stage Truncater: Stop the Reconciliation When Decode Success

Truncater is used to stop the reconciliation when the decoding of some stage is successful. 
It is created to reduce the time cost of the program, useless for the SIHO decoder, however.

In [8]:
from utils import control_identity, control_flusher
# control_identity will output the input data iff control is not 0, otherwise a zero vector will be output

# the first truncater truncates alice bits to 0 if last stage decodes successfully
truncater_alice_bits = []
# llr truncater (float) is used to truncate the llr inputs to the decoder
truncater_llr_stage = []
# frozen vector truncater (int) is used to truncate the frozen bits fed to the decoder
# truncater_frozen_vec_stage = []
# monitor truncater (int) is used to truncate the reference input of the monitor
truncater_mnt_input_stage = []
for stage in range(len(frozen_vec_stage)): 
    truncater_alice_bits.append(control_identity(block_encoded_bits, dtype = np.int32, control_dtype = np.int32))
    truncater_llr_stage.append(control_flusher(block_encoded_bits, dtype = np.float32, control_dtype = np.int32))
    # truncater_frozen_vec_stage.append(control_identity(block_encoded_bits, dtype = np.int32, control_dtype = np.int32))
    truncater_mnt_input_stage.append(control_identity(block_encoded_bits, dtype = np.int32, control_dtype = np.int32))
    pass 

### Initialize all Modules

After encoding Alice's sifted KEY to a codeword, a `mask_selector` will extract the effective frozen bits. 
The effective frozen bits will be used to generate the frozen vector by a `mask_fillor`, and then the shortener will modify the shortened bits' LLRs.  

In [9]:
from utils import BSC_digital_p, extractor, signal_waiter2, mask_fillor, mask_selector
from py_aff3ct.module.monitor import Monitor_BFER_AR

# In the following stages, the source will be truncated to allow more frozen bits, while keeping same sifted KEY
src_alice = py_aff3ct.module.source.Source_random_fast(len_sifted_key)
src_bob_from_alice = BSC_digital_p(q_ber_stat, len_sifted_key)
print("Simulated QBER: {}".format(q_ber_stat))

# The padder will fill the sifted KEY bits into non-shortened positions
non_shortened_mask = np.logical_not(np.asarray(shortened_vec, dtype=np.bool)).astype(np.int32).reshape((1, -1))
padder_alice = mask_fillor(len_sifted_key, block_encoded_bits, dtype=np.int32)
padder_bob = mask_fillor(len_sifted_key, block_encoded_bits, dtype=np.int32)

# A mask selector will be used to extract the info bits in the encoded alice vector
info_bits_selector_alice_stage = []
info_bits_mask_stage = []

# Monitor of the consistency between the decoding result and encoded result of padded alice KEY)
mnt_stage = []
# CASCL monitor watch the success of decoder - indicated by CRC check
mnt_cascl_stage = []

# We also need another selector to extract the frozen bits in the encoded alice vector
frozen_bits_selector_alice_stage = []
effective_frozen_vec_mask_stage = [deepcopy(bl.astype(np.int32).reshape((1, -1))) for bl in effective_frozen_vec_stage]
# frozen_bits_mask_stage = []
# Then a mask filor to fill the frozen bits into a all-zero vector to be provided to Bob's decoder
frozen_bits_fillor_alice_stage = []

# An extractor will be utilized to extract the crc value in output message of CRC "build" method
ext_crc_stage = []

# Wait until the crc const is set up
waiter_stage = []

mask_fillor_alice_stage = []
mask_selector_alice_stage = []
mask_fillor_bob_stage = []
for stage in range(len(frozen_vec_stage)): 

    info_bits_selector_alice_stage.append(mask_selector(block_encoded_bits, info_bits_stage[stage], dtype=np.int32))
    info_bits_mask_stage.append(deepcopy(np.logical_not(np.asarray(frozen_vec_stage[stage], dtype=np.bool)).astype(np.int32).reshape((1, -1))))

    mnt_stage.append(Monitor_BFER_AR(block_encoded_bits, num_samples, max_n_frames=num_samples))
    mnt_cascl_stage.append(Monitor_BFER_AR(1, num_samples, max_n_frames=num_samples))

    frozen_bits_selector_alice_stage.append(mask_selector(block_encoded_bits, effective_frozen_bits_stage[stage], dtype=np.int32))
    # frozen_bits_mask_stage.append(deepcopy(np.asarray(frozen_vec_stage[stage], dtype=np.bool)).astype(np.int32))
    frozen_bits_fillor_alice_stage.append(mask_fillor(effective_frozen_bits_stage[stage], block_encoded_bits, dtype=np.int32))

    ext_crc_stage.append(extractor(block_encoded_bits + crc_size, block_encoded_bits, block_encoded_bits + crc_size))
    # ext_crc_stage.append(extractor(info_bits_stage[stage] + crc_size, info_bits_stage[stage], info_bits_stage[stage] + crc_size))

    waiter_stage.append(signal_waiter2(block_encoded_bits, 1, in1_dtype=np.float32, in2_dtype=np.int32))

    pass

Simulated QBER: 0.03597984449309278


### Simulation: Randomly Sample Sifted KEYs and Test the Performance of AIR Scheme

**Note that, once a socket is bound to another socket, you cannot simply overwrite the binding relation.** Hence:

- If you want to reconcile real sifted KEY data, then jump to the **Experiment** part.
- If you want to test the codes, then jump to the **Step-by-step Test** part.

In [ ]:
src_bob_from_alice["digital_bit_flip ::input_bits"].bind(src_alice["generate ::U_K"])

padder_alice["fill ::mask"].bind(non_shortened_mask)
padder_alice["fill ::input"].bind(src_alice["generate ::U_K"])
padder_bob["fill ::mask"].bind(non_shortened_mask)
padder_bob["fill ::input"].bind(src_bob_from_alice["digital_bit_flip ::output_bits"])

# enc_alice["light_encode ::U_N"].bind(padder_alice["fill ::output"])

success_status = np.zeros(1, dtype = np.int32).reshape(1, -1)

for stage in range(len(frozen_vec_stage)):
    
    if stage == 0:
        enc_stage[stage]["light_encode ::U_N"].bind(padder_alice["fill ::output"])
    else:
        truncater_alice_bits[stage]["control_identity ::input"].bind(padder_alice["fill ::output"])
        truncater_alice_bits[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
        enc_stage[stage]["light_encode ::U_N"].bind(truncater_alice_bits[stage]["control_identity ::output"])
        pass

    # info_bits_selector_alice_stage[stage]["select ::input"].bind(enc_stage[stage]["light_encode ::X_N"])
    # info_bits_selector_alice_stage[stage]["select ::input"].bind(enc_alice["light_encode ::X_N"])
    # info_bits_selector_alice_stage[stage]["select ::mask"].bind(info_bits_mask_stage[stage])

    # # Feed the selected info bits to CRC checker
    # crc_stage[stage]["build ::U_K1"].bind(info_bits_selector_alice_stage[stage]["select ::output"])

    # Reuse the CRC tag in each stage
    crc_stage[stage]["build ::U_K1"].bind(enc_stage[stage]["light_encode ::X_N"])
    # if stage == 0:
    #     crc_stage[stage]["build ::U_K1"].bind(enc_alice["light_encode ::X_N"])
    # else:
    #     truncater_alice_bits[stage]["control_identity ::input"].bind(enc_alice["light_encode ::X_N"])
    #     truncater_alice_bits[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
    #     crc_stage[stage]["build ::U_K1"].bind(truncater_alice_bits[stage]["control_identity ::output"])
    
    
    # extract the crc string then feed to set_crc_const
    ext_crc_stage[stage]["extract ::input"].bind(crc_stage[stage]["build ::U_K2"])
    dec_stage[stage]["set_crc_const ::V_crc"].bind(ext_crc_stage[stage]["extract ::output"])
    # Synchronize the inputs
    waiter_stage[stage]["wait ::input_2"].bind(dec_stage[stage]["set_crc_const ::status"])

    # Select the frozen bits and send to decoder
    frozen_bits_selector_alice_stage[stage]["select ::input"].bind(enc_stage[stage]["light_encode ::X_N"])
    # frozen_bits_selector_alice_stage[stage]["select ::input"].bind(enc_alice["light_encode ::X_N"])
    frozen_bits_selector_alice_stage[stage]["select ::mask"].bind(effective_frozen_vec_mask_stage[stage])
    
    # Generate the frozen vector fed to decode_siho_cw_flexible_frozen
    frozen_bits_fillor_alice_stage[stage]["fill ::input"].bind(frozen_bits_selector_alice_stage[stage]["select ::output"])
    frozen_bits_fillor_alice_stage[stage]["fill ::mask"].bind(effective_frozen_vec_mask_stage[stage])

    # Compute the LLRs
    llr_bsc_stage[stage]["llr_bsc ::input"].bind(padder_bob["fill ::output"])

    # Then shorten the polar code by setting a large LLR on each shortened position
    shortener_stage[stage]["shorten ::input"].bind(llr_bsc_stage[stage]["llr_bsc ::output"])

    # Synchronize the inputs
    waiter_stage[stage]["wait ::input_1"].bind(shortener_stage[stage]["shorten ::output"])

    dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(frozen_bits_fillor_alice_stage[stage]["fill ::output"])

    # Decode
    if stage == 0:
        dec_stage[stage]["decode_siho_cw_flexible_frozen ::Y_N"].bind(waiter_stage[stage]["wait ::output"])
        # dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(frozen_bits_fillor_alice_stage[stage]["fill ::output"])

        mnt_stage[stage]["check_errors ::U"].bind(enc_stage[stage]["light_encode ::X_N"])
        # mnt_stage[stage]["check_errors ::U"].bind(enc_alice["light_encode ::X_N"])
        mnt_stage[stage]["check_errors ::V"].bind(dec_stage[stage]["decode_siho_cw_flexible_frozen ::V_N"])
    else:
        truncater_llr_stage[stage]["control_identity ::input"].bind(waiter_stage[stage]["wait ::output"])
        # truncater_frozen_vec_stage[stage]["control_identity ::input"].bind(frozen_bits_fillor_alice_stage[stage]["fill ::output"])
        truncater_mnt_input_stage[stage]["control_identity ::input"].bind(enc_stage[stage]["light_encode ::X_N"])
        # truncater_mnt_input_stage[stage]["control_identity ::input"].bind(enc_alice["light_encode ::X_N"])

        truncater_llr_stage[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
        # truncater_frozen_vec_stage[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
        truncater_mnt_input_stage[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])

        dec_stage[stage]["decode_siho_cw_flexible_frozen ::Y_N"].bind(truncater_llr_stage[stage]["control_identity ::output"])
        # dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(truncater_frozen_vec_stage[stage]["control_identity ::output"])

        mnt_stage[stage]["check_errors ::U"].bind(truncater_mnt_input_stage[stage]["control_identity ::output"])
        mnt_stage[stage]["check_errors ::V"].bind(dec_stage[stage]["decode_siho_cw_flexible_frozen ::V_N"])
        pass

    mnt_cascl_stage[stage]["check_errors ::U"].bind(success_status)
    mnt_cascl_stage[stage]["check_errors ::V"].bind(dec_stage[stage]["decode_siho_cw_flexible_frozen ::status"])
    pass

### Execute

In [ ]:
seq = py_aff3ct.tools.pipeline.Pipeline(
    [src_alice["generate"]], 
    [], 
    [
        [[src_alice["generate"]], [], [mnt_fr["check_errors"] for mnt_fr in mnt_stage] + [mnt_cascl["check_errors"] for mnt_cascl in mnt_cascl_stage]], 
        [[mnt_fr["check_errors"] for mnt_fr in mnt_stage] + [mnt_cascl["check_errors"] for mnt_cascl in mnt_cascl_stage], [], []]
    ], 
    [3, 1], # n_threads: more threads cost more cores and much more RAM
    [10], 
    [False]
)

l_tasks = seq.get_tasks_per_types()
for lt in l_tasks:
    for t in lt: 
        t.stats = True

# seeders = seq.get_modules_set_seed()
# for mdl in seeders:
#     mdl.set_seed(np.random.randint(seed, dtype=int))

for stage in range(len(mnt_stage)):
    mnt_stage[stage].reset()
    mnt_cascl_stage[stage].reset()

In [12]:
from utils import seq_execution

se = seq_execution(seq = seq)
se.start()

print("Total Frames | Frame Errors S(0) | Frame Errors S(-1) | Used Time | Remaining Time")

while se.is_alive():
    time.sleep(1)
    # Don't forget this time.sleep... otherwise the output could possibly crash the notebook

    elapsed = time.time() - se.starting_time
    total_fra = mnt_stage[-1].get_n_analyzed_fra()
    # Return the number of frame errors
    n_fe_0 = mnt_stage[0].get_n_fe()
    n_fe_l = mnt_stage[-1].get_n_fe()
    
    remaining = max((elapsed/max(total_fra, 1))*(num_samples-total_fra),0)
    print("%12d | %17d | %18d | %9s | %13s"%(total_fra, n_fe_0, n_fe_l, time.strftime('%H:%M:%S', time.gmtime(elapsed)), time.strftime('%H:%M:%S', time.gmtime(remaining))), end="\r")
    pass

Total Frames | Frame Errors S(0) | Frame Errors S(-1) | Used Time | Remaining Time


### Special Remark: Feature of py_aff3ct Monitor

When no error is observed, `monitor.get_fer()` will not return 0. 
Instead, it will return the rate corresponding to exactly 1 error. 
Same for `get_ber()`.
Hence, you should use `monitor.get_n_analyzed_fra()` and `monitor.get_n_fe()` to know how many frame errors occurred and what's the exactly error rate. 

In [13]:
average_efficiency = 0.
for stage in range(len(mnt_stage)):
    print("Stage {} with efficiency {}: Totally {} frames analyzed, {} frame error occurred, {} CA-SCL decoding failure observed.".format(stage, effective_efficiency_stage[stage] + crc_efficiency_gain, mnt_stage[stage].get_n_analyzed_fra(), mnt_stage[stage].get_n_fe(), mnt_cascl_stage[stage].get_n_fe()))
    print("Final BER: FR - {}; Final FER: FR - {}; CA-SCL decoding failure rate - {}".format(mnt_stage[stage].get_ber(), mnt_stage[stage].get_fer(), mnt_cascl_stage[stage].get_fer()))
    if stage == 0:
        average_efficiency += (effective_efficiency_stage[stage] + crc_efficiency_gain) * (num_samples - mnt_stage[stage].get_n_fe())
    else:
        average_efficiency += (effective_efficiency_stage[stage] + crc_efficiency_gain) * (mnt_stage[stage - 1].get_n_fe() - mnt_stage[stage].get_n_fe())
        pass
    pass
average_efficiency += (effective_efficiency_stage[-1] + crc_efficiency_gain) * mnt_stage[-1].get_n_fe()
average_efficiency = average_efficiency / num_samples
print("Average efficiency: {}".format(average_efficiency))

Stage 0 with efficiency 1.1799657981760419: Totally 1000 frames analyzed, 45 frame error occurred, 45 CA-SCL decoding failure observed.
Final BER: FR - 0.004726112354546785; Final FER: FR - 0.04500000178813934; CA-SCL decoding failure rate - 0.04500000178813934
Stage 1 with efficiency 1.1899698009723403: Totally 1000 frames analyzed, 20 frame error occurred, 20 CA-SCL decoding failure observed.
Final BER: FR - 0.002000099280849099; Final FER: FR - 0.019999999552965164; CA-SCL decoding failure rate - 0.019999999552965164
Stage 2 with efficiency 1.2099778065649367: Totally 1000 frames analyzed, 8 frame error occurred, 8 CA-SCL decoding failure observed.
Final BER: FR - 0.0007243347354233265; Final FER: FR - 0.00800000037997961; CA-SCL decoding failure rate - 0.00800000037997961
Stage 3 with efficiency 1.2499938177501297: Totally 1000 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Final BER: FR - 7.629394893626795e-09; Final FER: FR - 0.0010000000474974513; C

## Experiment: Reconcile Real Sifted KEY

**Configuration of Computation Graph**

The truncaters are disabled in this part to show the decode result of every stage.

In [10]:
reconcile_try = 1
# Re-initialize the mnt_stage and mnt_cascl
mnt_stage = []
mnt_cascl_stage = []

for stage in range(len(frozen_vec_stage)):
    mnt_stage.append(Monitor_BFER_AR(block_encoded_bits, reconcile_try, max_n_frames=reconcile_try))
    mnt_cascl_stage.append(Monitor_BFER_AR(1, reconcile_try, max_n_frames=reconcile_try))


# The src_alice["generate ::U_K"] should be replaced by Alice's sifted key from file
# The src_bob_from_alice["digital_bit_flip ::output_bits"] should be replaced by Bob's sifted key from file
fixed_alice_from_file = alice_sifted_key.reshape((1, -1)).astype(np.int32)
fixed_bob_from_file = bob_sifted_key.reshape((1, -1)).astype(np.int32)

# src_bob_from_alice["digital_bit_flip ::input_bits"].bind(src_alice["generate ::U_K"])

padder_alice["fill ::mask"].bind(non_shortened_mask)
padder_alice["fill ::input"].bind(fixed_alice_from_file)
# padder_alice["fill ::input"].bind(src_alice["generate ::U_K"])
padder_bob["fill ::mask"].bind(non_shortened_mask)
padder_bob["fill ::input"].bind(fixed_bob_from_file)

success_status = np.zeros(1, dtype = np.int32).reshape(1, -1)

for stage in range(len(frozen_vec_stage)):
    
    if True:
    # if stage == 0:
        enc_stage[stage]["light_encode ::U_N"].bind(padder_alice["fill ::output"])
    else:
        truncater_alice_bits[stage]["control_identity ::input"].bind(padder_alice["fill ::output"])
        truncater_alice_bits[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
        enc_stage[stage]["light_encode ::U_N"].bind(truncater_alice_bits[stage]["control_identity ::output"])
        pass

    # info_bits_selector_alice_stage[stage]["select ::input"].bind(enc_stage[stage]["light_encode ::X_N"])
    # info_bits_selector_alice_stage[stage]["select ::mask"].bind(info_bits_mask_stage[stage])

    # Reuse the CRC tag in each stage
    crc_stage[stage]["build ::U_K1"].bind(enc_stage[stage]["light_encode ::X_N"])
    # # Feed the selected info bits to CRC checker
    # crc_stage[stage]["build ::U_K1"].bind(info_bits_selector_alice_stage[stage]["select ::output"])

    # extract the crc string then feed to set_crc_const
    ext_crc_stage[stage]["extract ::input"].bind(crc_stage[stage]["build ::U_K2"])
    dec_stage[stage]["set_crc_const ::V_crc"].bind(ext_crc_stage[stage]["extract ::output"])
    # Synchronize the inputs
    waiter_stage[stage]["wait ::input_2"].bind(dec_stage[stage]["set_crc_const ::status"])

    # Select the frozen bits and send to decoder
    frozen_bits_selector_alice_stage[stage]["select ::input"].bind(enc_stage[stage]["light_encode ::X_N"])
    frozen_bits_selector_alice_stage[stage]["select ::mask"].bind(effective_frozen_vec_mask_stage[stage])
    
    # Generate the frozen vector fed to decode_siho_cw_flexible_frozen
    frozen_bits_fillor_alice_stage[stage]["fill ::input"].bind(frozen_bits_selector_alice_stage[stage]["select ::output"])
    frozen_bits_fillor_alice_stage[stage]["fill ::mask"].bind(effective_frozen_vec_mask_stage[stage])

    # Compute the LLRs
    llr_bsc_stage[stage]["llr_bsc ::input"].bind(padder_bob["fill ::output"])

    # Then shorten the polar code by setting a large LLR on each shortened position
    shortener_stage[stage]["shorten ::input"].bind(llr_bsc_stage[stage]["llr_bsc ::output"])

    # Synchronize the inputs
    waiter_stage[stage]["wait ::input_1"].bind(shortener_stage[stage]["shorten ::output"])

    dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(frozen_bits_fillor_alice_stage[stage]["fill ::output"])

    # Decode
    if True:
    # if stage == 0:
        dec_stage[stage]["decode_siho_cw_flexible_frozen ::Y_N"].bind(waiter_stage[stage]["wait ::output"])
        # dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(frozen_bits_fillor_alice_stage[stage]["fill ::output"])

        mnt_stage[stage]["check_errors ::U"].bind(enc_stage[stage]["light_encode ::X_N"])
        mnt_stage[stage]["check_errors ::V"].bind(dec_stage[stage]["decode_siho_cw_flexible_frozen ::V_N"])
    else:
        truncater_llr_stage[stage]["control_identity ::input"].bind(waiter_stage[stage]["wait ::output"])
        # truncater_frozen_vec_stage[stage]["control_identity ::input"].bind(frozen_bits_fillor_alice_stage[stage]["fill ::output"])
        truncater_mnt_input_stage[stage]["control_identity ::input"].bind(enc_stage[stage]["light_encode ::X_N"])

        truncater_llr_stage[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
        # truncater_frozen_vec_stage[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])
        truncater_mnt_input_stage[stage]["control_identity ::control"].bind(dec_stage[stage-1]["decode_siho_cw_flexible_frozen ::status"])

        dec_stage[stage]["decode_siho_cw_flexible_frozen ::Y_N"].bind(truncater_llr_stage[stage]["control_identity ::output"])
        # dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(truncater_frozen_vec_stage[stage]["control_identity ::output"])

        mnt_stage[stage]["check_errors ::U"].bind(truncater_mnt_input_stage[stage]["control_identity ::output"])
        mnt_stage[stage]["check_errors ::V"].bind(dec_stage[stage]["decode_siho_cw_flexible_frozen ::V_N"])
        pass

    mnt_cascl_stage[stage]["check_errors ::U"].bind(success_status)
    mnt_cascl_stage[stage]["check_errors ::V"].bind(dec_stage[stage]["decode_siho_cw_flexible_frozen ::status"])
    pass

**Execute**

In [11]:
seq = py_aff3ct.tools.pipeline.Pipeline(
    [padder_alice["fill"], padder_bob["fill"]], 
    [], 
    [
        [[padder_alice["fill"], padder_bob["fill"]], [], [mnt_fr["check_errors"] for mnt_fr in mnt_stage] + [mnt_cascl["check_errors"] for mnt_cascl in mnt_cascl_stage]], 
        [[mnt_fr["check_errors"] for mnt_fr in mnt_stage] + [mnt_cascl["check_errors"] for mnt_cascl in mnt_cascl_stage], [], []]
    ], 
    [1, 1], 
    [1], 
    [False]
)

In [12]:
from utils import seq_execution

l_tasks = seq.get_tasks_per_types()
for lt in l_tasks:
    for t in lt: 
        t.stats = True

# seeders = seq.get_modules_set_seed()
# for mdl in seeders:
#     mdl.set_seed(np.random.randint(seed, dtype=int))

for stage in range(len(mnt_stage)):
    mnt_stage[stage].reset()
    mnt_cascl_stage[stage].reset()
    pass

se = seq_execution(seq = seq)
se.start()

while se.is_alive():
    time.sleep(1)

    elapsed = time.time() - se.starting_time
    total_fra = mnt_stage[-1].get_n_analyzed_fra()
    pass

truncate_flag = 0
print("Configured QBER (Assumed QBER): {}".format(q_ber))
print("Statistical QBER from Data:     {}".format(q_ber_stat))
for stage in range(len(mnt_stage)):
    print("Stage {} with efficiency {}: Totally {} frames analyzed, {} frame error occurred, {} CA-SCL decoding failure observed.".format(stage, effective_efficiency_stage[stage] + crc_efficiency_gain, mnt_stage[stage].get_n_analyzed_fra(), mnt_stage[stage].get_n_fe(), mnt_cascl_stage[stage].get_n_fe()))
    if (mnt_cascl_stage[stage].get_n_fe() == 0) and (truncate_flag == 0):
        print("-------- The following stages will be truncated because the reconciliation is considered as successful.")
        truncate_flag = 1
        pass

Configured QBER (Assumed QBER): 0.0375
Statistical QBER from Data:     0.03597984449309278
Stage 0 with efficiency 1.1199839923805153: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 1 with efficiency 1.1219679085468697: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 2 with efficiency 1.123994035695487: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 3 with efficiency 1.1259779518618416: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 4 with efficiency 1.127961868028196: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 5 with efficiency 1.1299879951768135: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
-------- The following stages will be truncated because the reconciliation is considered as successful.
Stage 6 with efficien

#### Reconcile Real Sifted KEYs with Imperfect Assumed QBER

With assumed QBER $p_{QBER} = 0.0375$ and $p_{real} \approx 0.036$ from real sifted KEYs data, the following results can be reproduced:

```{text}
Configured QBER (Assumed QBER): 0.0375
Statistical QBER from Data:     0.03597984449309278
Stage 0 with efficiency 1.1199839923805153: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 1 with efficiency 1.1219679085468697: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 2 with efficiency 1.123994035695487: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 3 with efficiency 1.1259779518618416: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 4 with efficiency 1.127961868028196: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 5 with efficiency 1.1299879951768135: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
-------- The following stages will be truncated because the reconciliation is considered as successful.
Stage 6 with efficiency 1.1319719113431679: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 7 with efficiency 1.1339980384917854: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 8 with efficiency 1.1359819546581398: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Stage 9 with efficiency 1.1379658708244944: Totally 1 frames analyzed, 1 frame error occurred, 1 CA-SCL decoding failure observed.
Stage 10 with efficiency 1.1399919979731117: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Stage 11 with efficiency 1.1419759141394663: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Stage 12 with efficiency 1.1439598303058207: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Stage 13 with efficiency 1.145985957454438: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Stage 14 with efficiency 1.1479698736207926: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
Stage 15 with efficiency 1.1499960007694099: Totally 1 frames analyzed, 0 frame error occurred, 0 CA-SCL decoding failure observed.
```

Note that in order to reconcile the KEYs with better efficiency, the stages of efficiency is configured to be much finer than the one used in simulation. 
In simulation, such a refined stages of efficiency could consume to much computation power.

## Step-by-step Check of Simulation

In case anyone wants to check the codes or learn how the codes work. 
Here is a step-by-step check of the simulation.
For each module's tasks, the input and output sockets can be bound to a numpy array with corresponding shape and dtype. 
A numpy array can be bound to more than one socket, and hence one can use it to output the variables.
Also, you can use `module["name_of_task"].debug = True` to use py_aff3ct built-in debug info output method. 
However, for large-sized inputs and outputs it may make the notebook become laggy or crash. 

In [10]:
stage = 0

alice_bits = np.zeros((1, len_sifted_key), dtype = np.int32)
src_alice["generate ::U_K"].bind(alice_bits)
src_alice["generate"].exec()
print("alice bits:", alice_bits)

bob_bits = np.zeros((1, len_sifted_key), dtype = np.int32)
src_bob_from_alice["digital_bit_flip ::input_bits"].bind(alice_bits)
src_bob_from_alice["digital_bit_flip ::output_bits"].bind(bob_bits)
src_bob_from_alice["digital_bit_flip"].exec()
print("  bob bits:", bob_bits)
print("Differences:", np.logical_xor(alice_bits, bob_bits).astype(np.int32).sum())

padded_alice_bits = np.zeros((1, block_encoded_bits), dtype = np.int32)
padder_alice["fill ::mask"].bind(non_shortened_mask)
padder_alice["fill ::input"].bind(alice_bits)
padder_alice["fill ::output"].bind(padded_alice_bits)
padded_bob_bits = np.zeros((1, block_encoded_bits), dtype = np.int32)
padder_bob["fill ::mask"].bind(non_shortened_mask)
padder_bob["fill ::input"].bind(bob_bits)
padder_bob["fill ::output"].bind(padded_bob_bits)
padder_alice["fill"].exec()
padder_bob["fill"].exec()

print("padded alice bits:", padded_alice_bits)
print("padded   bob bits:", padded_bob_bits)
print("1s on shortened position:", padded_alice_bits[0, shortened_vec].sum(), padded_bob_bits[0, shortened_vec].sum(), )

encoded_alice = np.zeros_like(padded_alice_bits)
enc_stage[stage]["light_encode ::U_N"].bind(padded_alice_bits)
enc_stage[stage]["light_encode ::X_N"].bind(encoded_alice)
enc_stage[stage]["light_encode"].exec()

print("encoded alice bits:", encoded_alice)
print("1s on shortened position:", encoded_alice[0, shortened_vec].sum())
print("1s on info      position:", encoded_alice[np.where(info_bits_mask_stage[stage] == 1)].sum())

encoded_alice_info_bits = np.zeros((1, info_bits_stage[stage]), dtype = np.int32)
info_bits_selector_alice_stage[stage]["select ::input"].bind(encoded_alice)
info_bits_selector_alice_stage[stage]["select ::mask"].bind(info_bits_mask_stage[stage])
info_bits_selector_alice_stage[stage]["select ::output"].bind(encoded_alice_info_bits)
info_bits_selector_alice_stage[stage]["select"].exec()

print("encoded alice info bits:", encoded_alice_info_bits)
print("1s:", encoded_alice_info_bits.sum())

# encoded_alice_crc = np.zeros((1, info_bits_stage[stage] + crc_size), dtype=np.int32)
encoded_alice_crc = np.zeros((1, block_encoded_bits + crc_size), dtype=np.int32)
# crc_stage[stage]["build ::U_K1"].bind(encoded_alice_info_bits)
crc_stage[stage]["build ::U_K1"].bind(encoded_alice)
crc_stage[stage]["build ::U_K2"].bind(encoded_alice_crc)
crc_stage[stage]["build"].exec()

print("encoded alice info bits with crc tag:", encoded_alice_crc)
print("numpy crc tag bits:", encoded_alice_crc[:, -crc_size:])

alice_crc_tag = np.zeros((1, crc_size), dtype = np.int32)
ext_crc_stage[stage]["extract ::input"].bind(encoded_alice_crc)
ext_crc_stage[stage]["extract ::output"].bind(alice_crc_tag)
ext_crc_stage[stage]["extract"].exec()

print("extracted crc tag:", alice_crc_tag)
if np.logical_xor(alice_crc_tag, encoded_alice_crc[:, -crc_size:]).sum():
    print("CRC Tag Extractor Error!")
else:
    print("CRC Tag Extractor Functioning Properly.")

set_crc_const_status = np.ones((1, 1), dtype = np.int32)
dec_stage[stage]["set_crc_const ::V_crc"].bind(alice_crc_tag) 
dec_stage[stage]["set_crc_const ::status"].bind(set_crc_const_status)
dec_stage[stage]["set_crc_const"].exec()

print("set_crc_const executed, status:", set_crc_const_status)

waiter_stage[stage]["wait ::input_2"].bind(set_crc_const_status)

encoded_alice_frozen_bits = np.zeros((1, effective_frozen_vec_stage[stage].astype(np.int32).sum()), dtype = np.int32)
frozen_bits_selector_alice_stage[stage]["select :: input"].bind(encoded_alice)
frozen_bits_selector_alice_stage[stage]["select :: mask"].bind(effective_frozen_vec_mask_stage[stage])
frozen_bits_selector_alice_stage[stage]["select :: output"].bind(encoded_alice_frozen_bits)
frozen_bits_selector_alice_stage[stage]["select"].exec()

print("encoded alice frozen bits:   ", encoded_alice_frozen_bits)
print("np encoded alice frozen bits:", encoded_alice[0, np.where(effective_frozen_vec_mask_stage[stage].astype(np.bool))[1]])
if np.logical_xor(encoded_alice_frozen_bits, encoded_alice[np.where(effective_frozen_vec_mask_stage[stage].astype(np.bool))]).sum(): 
    print("select encoded alice codeword effective frozen vector error!")
else:
    print("select encoded alice codeword effective frozen bits properly.")
    pass


bob_decode_frozen_vector = np.zeros((1, block_encoded_bits), dtype = np.int32)
frozen_bits_fillor_alice_stage[stage]["fill ::input"].bind(encoded_alice_frozen_bits)
frozen_bits_fillor_alice_stage[stage]["fill ::mask"].bind(effective_frozen_vec_mask_stage[stage])
frozen_bits_fillor_alice_stage[stage]["fill ::output"].bind(bob_decode_frozen_vector)
frozen_bits_fillor_alice_stage[stage]["fill"].exec()

print("Bob decoding frozen vector - from alice:", bob_decode_frozen_vector)
print("1s                   :", bob_decode_frozen_vector.sum())
print("1s on frozen position:", bob_decode_frozen_vector[np.where(effective_frozen_vec_mask_stage[stage] == 1)].sum())
if np.logical_xor(bob_decode_frozen_vector[np.where(effective_frozen_vec_mask_stage[stage] == 1)], 
                  encoded_alice_frozen_bits).sum():
    print("fill encoded alice codeword effective frozen bits error!")
else: 
    print("fill encoded alice codeword effective frozen bits properly.")

bob_decode_llr = np.zeros((1, block_encoded_bits), dtype = np.float32)
llr_bsc_stage[stage]["llr_bsc ::input"].bind(padded_bob_bits)
llr_bsc_stage[stage]["llr_bsc ::output"].bind(bob_decode_llr)
llr_bsc_stage[stage]["llr_bsc"].exec()

print("Bob decoding llr vector - from padded bob sifted KEY:", bob_decode_llr)
print("number of maximum values:", np.where(bob_decode_llr == bob_decode_llr.max())[0].shape[0])
print("number of minimum values:", np.where(bob_decode_llr == bob_decode_llr.min())[0].shape[0])
print("sum (should equal the length N of polar code):", np.where(bob_decode_llr == bob_decode_llr.max())[0].shape[0] + np.where(bob_decode_llr == bob_decode_llr.min())[0].shape[0])

shortened_llr = np.zeros_like(bob_decode_llr)
shortener_stage[stage]["shorten ::input"].bind(bob_decode_llr)
shortener_stage[stage]["shorten ::output"].bind(shortened_llr)
shortener_stage[stage]["shorten"].exec()

print("Bob decoding shortened llr vector:", shortened_llr)
print("number of shortened values - 0 (should equal the number of shortened positions):", np.where(shortened_llr == shortened_llr.max())[0].shape[0])
print("number of shortened values - 1 (should equal 0                                ):", np.where(shortened_llr == -shortened_llr.max())[0].shape[0])
print("number of 0 values:", np.where(shortened_llr == bob_decode_llr.max())[0].shape[0])
print("number of 1 values:", np.where(shortened_llr == bob_decode_llr.min())[0].shape[0])
print("sum (should equal the length N of polar code):", 
    np.where(shortened_llr == shortened_llr.max())[0].shape[0] + \
        np.where(shortened_llr == -shortened_llr.max())[0].shape[0] + \
            np.where(shortened_llr == bob_decode_llr.max())[0].shape[0] + \
                np.where(shortened_llr == bob_decode_llr.min())[0].shape[0])

input_llr_dec_stage = np.zeros_like(shortened_llr)
waiter_stage[stage]["wait ::input_1"].bind(shortened_llr)
waiter_stage[stage]["wait ::output"].bind(input_llr_dec_stage)
waiter_stage[stage]["wait"].exec()
if np.abs(input_llr_dec_stage - shortened_llr).sum() < 1e-10:
    print("waiter operates properly.")
else:
    print("waiting operation error!")

dec_status = np.ones((1, 1), dtype = np.int32)
decoded_bob = np.zeros((1, block_encoded_bits), dtype = np.int32)
dec_stage[stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(bob_decode_frozen_vector)
dec_stage[stage]["decode_siho_cw_flexible_frozen ::Y_N"].bind(input_llr_dec_stage)
dec_stage[stage]["decode_siho_cw_flexible_frozen ::status"].bind(dec_status)
dec_stage[stage]["decode_siho_cw_flexible_frozen ::V_N"].bind(decoded_bob)
dec_stage[stage]["decode_siho_cw_flexible_frozen"].exec()
print("Decode status:", dec_status)
print("Status 0 for success, -1 for failure. It is common to have a failure result for low efficiency reconciliation.")

monitor_check_status = np.zeros((1, 1), dtype = np.int32)
mnt_stage[stage]["check_errors ::U"].bind(encoded_alice)
mnt_stage[stage]["check_errors ::V"].bind(decoded_bob)
mnt_stage[stage]["check_errors ::status"].bind(monitor_check_status)
mnt_stage[stage].reset()
mnt_stage[stage]["check_errors"].exec()

print("monitor check status:", monitor_check_status)

alice bits: [[0 1 1 ... 0 0 1]]
  bob bits: [[0 1 1 ... 0 0 0]]
Differences: 3770
padded alice bits: [[0 1 1 ... 0 1 0]]
padded   bob bits: [[0 1 1 ... 0 0 0]]
1s on shortened position: 0 0
encoded alice bits: [[0 1 1 ... 0 1 0]]
1s on shortened position: 0
1s on info      position: 38789
encoded alice info bits: [[0 1 1 ... 1 0 1]]
1s: 38789
encoded alice info bits with crc tag: [[0 1 1 ... 1 0 0]]
numpy crc tag bits: [[0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0]]
extracted crc tag: [[0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0]]
CRC Tag Extractor Functioning Properly.
set_crc_const executed, status: [[0]]
encoded alice frozen bits:    [[0 1 1 ... 0 0 1]]
np encoded alice frozen bits: [0 1 1 ... 0 0 1]
select encoded alice codeword effective frozen bits properly.
Bob decoding frozen vector - from alice: [[0 1 1 ... 0 0 0]]
1s                   : 14027
1s on frozen position: 14027
fill encoded alice codeword effective frozen bits properly.
Bob d

### Step-by-step Test of Simulation: Truncaters in Multi-stage IR

The following codes are for testing whether the truncaters are working properly or not. 
Ideally, the truncater will truncates the integer bits to 0 (and truncates the LLRs to 1) if the status of CA-SCL decoder is '0', which represents the success of decoding and gurantees that decoding process in the following stages are trivial. 

If you want to make sure the decoding process is properly impelemented in the next stage, you can configure an abnormally low efficiency (smaller than 1, for example) in the previous stage and make the decoding fail. 
Then, the truncater should not truncates any information and the reconciliation will be conducted with higher efficiency in this stage.

In [11]:
next_stage = stage + 1

truncater_alice_bits_output = np.zeros((1, block_encoded_bits), dtype = np.int32)
truncater_alice_bits[next_stage]["control_identity ::input"].bind(padded_alice_bits)
truncater_alice_bits[next_stage]["control_identity ::control"].bind(dec_status)
truncater_alice_bits[next_stage]["control_identity ::output"].bind(truncater_alice_bits_output)
truncater_alice_bits[next_stage]["control_identity"].exec()

print("Controlled by decode status {} in last stage, truncated alice bits: {}".format(dec_status, truncater_alice_bits_output))
print("1s:", truncater_alice_bits_output.sum())

if (not (truncater_alice_bits_output.sum() or dec_status[0, 0])) or \
    (truncater_alice_bits_output.sum() and dec_status[0, 0]):
    print("alice bits truncater functioning properly.")
else:
    print("alice bits truncater error!")

nxt_encoded_alice = np.zeros((1, block_encoded_bits), dtype = np.int32)
enc_stage[next_stage]["light_encode ::U_N"].bind(truncater_alice_bits_output)
enc_stage[next_stage]["light_encode ::X_N"].bind(nxt_encoded_alice)
enc_stage[next_stage]["light_encode"].exec()

print("Next stage encoded alice bits:", nxt_encoded_alice)
print("1s:", nxt_encoded_alice.sum())

nxt_encoded_alice_info_bits = np.zeros((1, info_bits_stage[next_stage]), dtype = np.int32)
info_bits_selector_alice_stage[next_stage]["select ::input"].bind(nxt_encoded_alice)
info_bits_selector_alice_stage[next_stage]["select ::mask"].bind(info_bits_mask_stage[next_stage])
info_bits_selector_alice_stage[next_stage]["select ::output"].bind(nxt_encoded_alice_info_bits)
info_bits_selector_alice_stage[next_stage]["select"].exec()
print("Next stage encoded alice info bits:", nxt_encoded_alice_info_bits)
print("1s:", nxt_encoded_alice_info_bits.sum())


# nxt_encoded_alice_crc = np.zeros((1, info_bits_stage[next_stage] + crc_size), dtype=np.int32)
nxt_encoded_alice_crc = np.zeros((1, block_encoded_bits + crc_size), dtype=np.int32)
# crc_stage[next_stage]["build ::U_K1"].bind(nxt_encoded_alice_info_bits)
crc_stage[next_stage]["build ::U_K1"].bind(nxt_encoded_alice)
crc_stage[next_stage]["build ::U_K2"].bind(nxt_encoded_alice_crc)
crc_stage[next_stage]["build"].exec()

print("encoded alice info bits with crc tag:", nxt_encoded_alice_crc)
print("numpy crc tag bits:", nxt_encoded_alice_crc[:, -crc_size:])

nxt_alice_crc_tag = np.zeros((1, crc_size), dtype = np.int32)
ext_crc_stage[next_stage]["extract ::input"].bind(nxt_encoded_alice_crc)
ext_crc_stage[next_stage]["extract ::output"].bind(nxt_alice_crc_tag)
ext_crc_stage[next_stage]["extract"].exec()

print("extracted crc tag:", nxt_alice_crc_tag)
if np.logical_xor(nxt_alice_crc_tag, nxt_encoded_alice_crc[:, -crc_size:]).sum():
    print("CRC Tag Extractor Error!")
else:
    print("CRC Tag Extractor Functioning Properly.")


nxt_set_crc_const_status = np.ones((1, 1), dtype = np.int32)
dec_stage[next_stage]["set_crc_const ::V_crc"].bind(nxt_alice_crc_tag) 
dec_stage[next_stage]["set_crc_const ::status"].bind(nxt_set_crc_const_status)
dec_stage[next_stage]["set_crc_const"].exec()

print("set_crc_const executed, status:", nxt_set_crc_const_status)

waiter_stage[next_stage]["wait ::input_2"].bind(nxt_set_crc_const_status)

nxt_encoded_alice_frozen_bits = np.zeros((1, effective_frozen_vec_stage[next_stage].astype(np.int32).sum()), dtype = np.int32)
frozen_bits_selector_alice_stage[next_stage]["select :: input"].bind(nxt_encoded_alice)
frozen_bits_selector_alice_stage[next_stage]["select :: mask"].bind(effective_frozen_vec_mask_stage[next_stage])
frozen_bits_selector_alice_stage[next_stage]["select :: output"].bind(nxt_encoded_alice_frozen_bits)
frozen_bits_selector_alice_stage[next_stage]["select"].exec()

print("encoded alice frozen bits:   ", nxt_encoded_alice_frozen_bits)
print("np encoded alice frozen bits:", nxt_encoded_alice[0, np.where(effective_frozen_vec_mask_stage[next_stage].astype(np.bool))[1]])
if np.logical_xor(nxt_encoded_alice_frozen_bits, nxt_encoded_alice[np.where(effective_frozen_vec_mask_stage[next_stage].astype(np.bool))]).sum(): 
    print("select encoded alice codeword effective frozen vector error!")
else:
    print("select encoded alice codeword effective frozen bits properly.")
    pass


nxt_bob_decode_frozen_vector = np.zeros((1, block_encoded_bits), dtype = np.int32)
frozen_bits_fillor_alice_stage[next_stage]["fill ::input"].bind(nxt_encoded_alice_frozen_bits)
frozen_bits_fillor_alice_stage[next_stage]["fill ::mask"].bind(effective_frozen_vec_mask_stage[next_stage])
frozen_bits_fillor_alice_stage[next_stage]["fill ::output"].bind(nxt_bob_decode_frozen_vector)
frozen_bits_fillor_alice_stage[next_stage]["fill"].exec()

print("Bob decoding frozen vector - from alice:", nxt_bob_decode_frozen_vector)
print("1s                   :", nxt_bob_decode_frozen_vector.sum())
print("1s on frozen position:", nxt_bob_decode_frozen_vector[np.where(effective_frozen_vec_mask_stage[next_stage] == 1)].sum())
if np.logical_xor(nxt_bob_decode_frozen_vector[np.where(effective_frozen_vec_mask_stage[next_stage] == 1)], 
                  nxt_encoded_alice_frozen_bits).sum():
    print("fill encoded alice codeword effective frozen bits error!")
else: 
    print("fill encoded alice codeword effective frozen bits properly.")


nxt_bob_decode_llr = np.zeros((1, block_encoded_bits), dtype = np.float32)
llr_bsc_stage[next_stage]["llr_bsc ::input"].bind(padded_bob_bits)
llr_bsc_stage[next_stage]["llr_bsc ::output"].bind(nxt_bob_decode_llr)
llr_bsc_stage[next_stage]["llr_bsc"].exec()

print("Bob decoding llr vector - from padded bob sifted KEY:", nxt_bob_decode_llr)
print("number of maximum values:", np.where(nxt_bob_decode_llr == nxt_bob_decode_llr.max())[0].shape[0])
print("number of minimum values:", np.where(nxt_bob_decode_llr == nxt_bob_decode_llr.min())[0].shape[0])
print("sum (should equal the length N of polar code):", np.where(nxt_bob_decode_llr == nxt_bob_decode_llr.max())[0].shape[0] + np.where(nxt_bob_decode_llr == nxt_bob_decode_llr.min())[0].shape[0])

nxt_shortened_llr = np.zeros_like(nxt_bob_decode_llr)
shortener_stage[next_stage]["shorten ::input"].bind(nxt_bob_decode_llr)
shortener_stage[next_stage]["shorten ::output"].bind(nxt_shortened_llr)
shortener_stage[next_stage]["shorten"].exec()

print("Bob decoding shortened llr vector:", nxt_shortened_llr)
print("number of shortened values - 0 (should equal the number of shortened positions):", np.where(nxt_shortened_llr == nxt_shortened_llr.max())[0].shape[0])
print("number of shortened values - 1 (should equal 0                                ):", np.where(nxt_shortened_llr == -nxt_shortened_llr.max())[0].shape[0])
print("number of 0 values:", np.where(nxt_shortened_llr == nxt_bob_decode_llr.max())[0].shape[0])
print("number of 1 values:", np.where(nxt_shortened_llr == nxt_bob_decode_llr.min())[0].shape[0])
print("sum (should equal the length N of polar code):", 
    np.where(nxt_shortened_llr == nxt_shortened_llr.max())[0].shape[0] + \
        np.where(nxt_shortened_llr == -nxt_shortened_llr.max())[0].shape[0] + \
            np.where(nxt_shortened_llr == nxt_bob_decode_llr.max())[0].shape[0] + \
                np.where(nxt_shortened_llr == nxt_bob_decode_llr.min())[0].shape[0])


nxt_input_llr_dec_stage = np.zeros_like(nxt_shortened_llr)
waiter_stage[next_stage]["wait ::input_1"].bind(nxt_shortened_llr)
waiter_stage[next_stage]["wait ::output"].bind(nxt_input_llr_dec_stage)
waiter_stage[next_stage]["wait"].exec()
if np.abs(nxt_input_llr_dec_stage - nxt_shortened_llr).sum() < 1e-10:
    print("waiter operates properly.")
else:
    print("waiting operation error!")


truncater_llr_bob_output = np.zeros((1, block_encoded_bits), dtype = np.float32)
truncater_llr_stage[stage]["control_identity ::input"].bind(nxt_input_llr_dec_stage)
truncater_llr_stage[stage]["control_identity ::control"].bind(dec_status)
truncater_llr_stage[stage]["control_identity ::output"].bind(truncater_llr_bob_output)
truncater_llr_stage[stage]["control_identity"].exec()

print("Controlled by status {}, output is {}".format(dec_status, truncater_llr_bob_output))
print("sum:", np.abs(truncater_llr_bob_output).sum())
print("When the bits are truncated to 0s, the llr should be set to (1.0)")

nxt_dec_status = np.ones((1, 1), dtype = np.int32)
nxt_decoded_bob = np.zeros((1, block_encoded_bits), dtype = np.int32)
dec_stage[next_stage]["decode_siho_cw_flexible_frozen ::F_N"].bind(nxt_bob_decode_frozen_vector)
dec_stage[next_stage]["decode_siho_cw_flexible_frozen ::Y_N"].bind(truncater_llr_bob_output)
dec_stage[next_stage]["decode_siho_cw_flexible_frozen ::status"].bind(nxt_dec_status)
dec_stage[next_stage]["decode_siho_cw_flexible_frozen ::V_N"].bind(nxt_decoded_bob)
dec_stage[next_stage]["decode_siho_cw_flexible_frozen"].exec()
print("Decode status:", nxt_dec_status)
print("Status 0 for success, -1 for failure. Note that for a trivial all-0 input the decode should always success.")

truncater_mnt_input_output = np.zeros((1, block_encoded_bits), dtype = np.int32)
truncater_mnt_input_stage[next_stage]["control_identity ::input"].bind(nxt_encoded_alice)
truncater_mnt_input_stage[next_stage]["control_identity ::control"].bind(dec_status)
truncater_mnt_input_stage[next_stage]["control_identity ::output"].bind(truncater_mnt_input_output)
truncater_mnt_input_stage[next_stage]["control_identity"].exec()

print("Controlled by status {}, the output if truncater_mnt_input is {}".format(dec_status, truncater_mnt_input_output))
print("1s:", truncater_mnt_input_output.sum())

nxt_monitor_check_status = np.zeros((1, 1), dtype = np.int32)
mnt_stage[next_stage]["check_errors ::U"].bind(truncater_mnt_input_output)
mnt_stage[next_stage]["check_errors ::V"].bind(nxt_decoded_bob)
mnt_stage[next_stage]["check_errors ::status"].bind(nxt_monitor_check_status)
mnt_stage[next_stage].reset()
mnt_stage[next_stage]["check_errors"].exec()

print("monitor check status:", nxt_monitor_check_status)

Controlled by decode status [[-1]] in last stage, truncated alice bits: [[0 1 1 ... 0 1 0]]
1s: 53150
alice bits truncater functioning properly.
Next stage encoded alice bits: [[0 1 1 ... 0 1 0]]
1s: 52816
Next stage encoded alice info bits: [[1 1 1 ... 1 0 1]]
1s: 37977
encoded alice info bits with crc tag: [[0 1 1 ... 1 0 0]]
numpy crc tag bits: [[0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0]]
extracted crc tag: [[0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0]]
CRC Tag Extractor Functioning Properly.
set_crc_const executed, status: [[0]]
encoded alice frozen bits:    [[0 1 1 ... 0 0 1]]
np encoded alice frozen bits: [0 1 1 ... 0 0 1]
select encoded alice codeword effective frozen bits properly.
Bob decoding frozen vector - from alice: [[0 1 1 ... 0 0 0]]
1s                   : 14839
1s on frozen position: 14839
fill encoded alice codeword effective frozen bits properly.
Bob decoding llr vector - from padded bob sifted KEY: [[ 3.2875724 -3.2875724 